In [1]:
import re
import fiona
import geopandas as gpd
import pandas as pd
from fiona import errors
from pandas import errors
from shapely.geometry import Point

In [2]:
from math import ceil, floor

from pyproj import Proj
from rasterio import open as rasopen
from rasterio.crs import CRS
from fiona import open as fopen
import fiona

# replace code below with 
from bounds import BBox, GeoBounds, RasterBounds

class BBox(object):
    def __init__(self):
        self.west = None
        self.east = None
        self.north = None
        self.south = None

    def as_tuple(self, order='wsen'):
        """ Find 4-tuple of extent
        :param order: order of cardinal directions, default='wsen'
        :return: 4-Tuple
        """
        if order == 'wsen':
            return self.west, self.south, self.east, self.north
        elif order == 'swne':
            return self.south, self.west, self.north, self.east
        elif order == 'nsew':
            return self.north, self.south, self.east, self.west

    def to_web_mercator(self):
        in_proj = Proj({'init': 'epsg:3857'})
        w, s = in_proj(self.west, self.south)
        e, n = in_proj(self.east, self.north)
        return w, s, e, n

    def to_mt_sp(self):
        in_proj = Proj('+proj=lcc +lat_1=45 +lat_2=49 +lat_0=44.25 +'
                       'lon_0=-109.5 +x_0=600000 +y_0=LE07_clip_L1TP_039027_20150529_20160902_01_T1_B1.TIF +ellps=GRS80 +units=m +no_defs')
        w, s = in_proj(self.west, self.south)
        e, n = in_proj(self.east, self.north)
        return w, s, e, n

    def to_epsg(self, epsg):
        in_proj = Proj({'init': 'epsg:{}'.format(epsg)})
        w, s = in_proj(self.west, self.south)
        e, n = in_proj(self.east, self.north)
        return w, s, e, n

    def to_geographic(self, epsg):
        in_proj = Proj({'init': 'epsg:{}'.format(epsg)})

        w, s = in_proj(self.west, self.south, inverse=True)

        e, n = in_proj(self.east, self.north, inverse=True)

        return w, s, e, n

    def to_lambert_conformal_conic(self):
        in_proj = Proj('+ellps=GRS80 +lat_0=23 +lat_1=29.5 +lat_2=45.5 +lon_0=-96 +no_defs +proj=aea'
                       ' +towgs84=0,0,0,0,0,0,0 +units=m +x_0=0 +y_0=0')
        w, s = in_proj(self.west, self.south)
        e, n = in_proj(self.east, self.north)
        return w, s, e, n

    def expand(self, **delta):

        if not delta:
            if self.west < 0:
                self.west = floor(self.west)
                self.east = ceil(self.east)
            else:
                self.west = ceil(self.west)
                self.east = floor(self.east)

            if self.north > 0:
                self.north = ceil(self.north)
                self.south = floor(self.south)
            else:
                self.north = floor(self.north)
                self.south = ceil(self.south)

        else:

            self.west += delta['west']
            self.east += delta['east']
            self.north += delta['north']
            self.south += delta['south']


class GeoBounds(BBox):
    """Spatial bounding box


    """

    def __init__(self, west=None, south=None, east=None, north=None, wsen=None):
        BBox.__init__(self)

        if wsen:
            self.west = wsen[0]
            self.south = wsen[1]
            self.east = wsen[2]
            self.north = wsen[3]
        else:
            self.west = west
            self.south = south
            self.east = east
            self.north = north


class RasterBounds(BBox):
    """ Spatial bounding box from raster extent.

    :param raster

    """

    def __init__(self, raster=None, affine_transform=None, profile=None, latlon=True):
        BBox.__init__(self)

        if raster:
            with rasopen(raster, 'r') as src:
                profile = src.profile
                affine = profile['transform']

        if affine_transform:
            affine = affine_transform

        col, row = 0, 0
        w, n = affine * (col, row)
        col, row = profile['width'], profile['height']
        e, s = affine * (col, row)

        if latlon and profile['crs'] != CRS({'init': 'epsg:4326'}):
            in_proj = Proj(init=profile['crs']['init'])
            self.west, self.north = in_proj(w, n, inverse=True)
            self.east, self.south = in_proj(e, s, inverse=True)

        else:
            self.north, self.west, self.south, self.east = n, w, s, e

    def get_nwse_tuple(self):
        return self.north, self.west, self.south, self.east

class VectorBounds(BBox):
    """ Spatial bounding box from vector extent.

    :param vector

    """

    def __init__(self, vector=None, profile=None, latlon=True):
        BBox.__init__(self)

        if vector:
            with fopen(vector, 'r') as src:
                self.crs = src.crs
                self.epsg = int(src.crs['init'].split(":")[1])
                self.profile = src.profile
                self.meta = src.meta
                self.west, self.south, self.east, self.north = src.bounds

        if latlon and self.crs != {'init': 'epsg:4326'}:
            in_proj = Proj(init=self.profile['crs']['init'])
            self.west, self.north = in_proj(self.west, self.north, inverse=True)
            self.east, self.south = in_proj(self.east, self.south, inverse=True)

        else:
            pass


if __name__ == '__main__':
    pass
# ========================= EOF ====================================================================

It would be good to get all of your code into a single file that can be imported into the notebook. I would move this notebook into the top level directory. Then you can import your functions from `get_extent.py`. ...Though, you'll have to change your import statement from `bounds`.

maybe like:
`from bounds import *`

In [29]:
from get_extent import *

ImportError: cannot import name VectorBounds

In [3]:
def get_extent_vector(shapefile):
    """

    Parameters
    ----------
    shapefile : str
        Filepath to a shapefile

    Returns
    -------
    Bounding box of vector in geographic coordinates

    """
    vb = VectorBounds(shapefile)
    bounds = list((vb.west, vb.south, vb.east, vb.north))
    return bounds

In [4]:

def get_extent_raster(raster):
    """

    Parameters
    ----------
    raster : str
        Filepath to a raster

    Returns
    -------
    Bounding box of raster in geographic coordinates

    """
    rbnds = RasterBounds(raster)
    bounds = list((rbnds.west, rbnds.south, rbnds.east, rbnds.north))

    return bounds

In [5]:
def get_extent_csv(csv):
    """

    Parameters
    ----------
    csv : str
        Filepath to a csv

    Returns
    -------
    Bounding box of csv if lat and lon exist in column names

    """
    csv_df = pd.read_csv(csv)

    class MyException(Exception):
        pass

    def generic_check_for_cols(csv_df, col_string):
        results = []
        pat = re.compile(col_string, flags=re.IGNORECASE)
        for col_name in csv_df.columns:
            if pat.match(col_name) is not None:
                results.append(col_name)
        if len(results) != 1:
            raise MyException
        return results

    def check_lat(csv_df):
        return generic_check_for_cols(csv_df, "lat")

    def check_lon(csv_df):
        return generic_check_for_cols(csv_df, "lon")

    try:
        lat_name = check_lat(csv_df)
        lon_name = check_lon(csv_df)
    except MyException:
        raise MyException('Latitude or longitude cannot be guessed.')

    csv_df['geometry'] = csv_df.apply(lambda r: Point(r[lon_name], r[lat_name]), axis=1)
    pnt_gdf = gpd.GeoDataFrame(csv_df)

    return pnt_gdf.total_bounds.tolist()


In [6]:
def get_extent(file_path):
    """

    Parameters
    ----------
    vector, raster or csv : str
        Filepath to vector, raster or csv

    Returns
    -------
    Bounding box of vector, raster or csv
    """
    try:
        extent = get_extent_csv(file_path)
    except pd.errors.ParserError:
        try:
            extent = get_extent_vector(file_path)
        except fiona.errors.FionaValueError:
            extent = get_extent_raster(file_path)
    return extent


In [11]:
herd_shp = "https://github.com/mariejohnson/spatial_introspect/raw/master/test_data/HerdSpatialDistribution/HerdSpatialDistribution.shp"
# can't get the shapefile to download via github but it works locally
herd_shp_local = "../test_data/HerdSpatialDistribution/HerdSpatialDistribution.shp"
CA_raster = "https://raw.github.com/mariejohnson/spatial_introspect/raw/master/test_data/NE1_50M_SR_W_tenth_CA.tif"
# same issue as above
CA_raster_local = "../test_data/NE1_50M_SR_W_tenth_CA.tif"
csv_no_geo = "https://raw.githubusercontent.com/mariejohnson/spatial_introspect/master/test_data/no_geo.csv"
csv_geo = "https://raw.githubusercontent.com/mariejohnson/spatial_introspect/master/test_data/SitiosMuestreoPasto.csv"

In [10]:
get_extent(herd_shp_local)

[81.61236877710331, 72.59573070400164, 90.77138370103438, 73.27285311714067]

In [12]:
get_extent(CA_raster_local)

[-125.99999999999997, 31.0, -112.99999999999997, 43.0]

In [13]:
get_extent(csv_geo)

[-77.05520652, -10.45003279, -76.78861605, -10.15487673]

In [14]:
get_extent(csv_no_geo)

MyException: Latitude or longitude cannot be guessed.

To install `bounds` (installing this way with the `!` may just do a temporary install:

In [18]:
!pip2 install --user git+https://github.com/dgketchum/bounds.git

  Cloning https://github.com/dgketchum/bounds.git to /private/var/folders/57/7c_cr6g56g13tmy0c9dqx1x00000gn/T/pip-req-build-5_BtMc
  Running setup.py bdist_wheel for bounds ... done
  Stored in directory: /private/var/folders/57/7c_cr6g56g13tmy0c9dqx1x00000gn/T/pip-ephem-wheel-cache-jMXfe3/wheels/cd/78/bc/365f786c0402f63e10e8ae64e4f539d15d6d8b019a95fab19b
Successfully built bounds


To do a regular install do this on the command line:

`pip2 install --user git+https://github.com/dgketchum/bounds.git`

or, to install for python 3, use `pip3`:

`pip3 install --user git+https://github.com/dgketchum/bounds.git`